In [49]:
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

##### ORIGINAL STEPS TO READ IN ALVARO CSVS

In [31]:
# df15 = pd.read_csv('../clean_csv_files/alvaro_15.csv')
# df16 = pd.read_csv('../clean_csv_files/alvaro_16.csv')
# df17 = pd.read_csv('../clean_csv_files/alvaro_17.csv')
# df18 = pd.read_csv('../clean_csv_files/alvaro_18.csv')
# df19 = pd.read_csv('../clean_csv_files/alvaro_19.csv')

In [32]:
# df_combined_1 = pd.concat([df15, df16, df17, df18, df19])
# df_combined_1.info()

In [33]:
# df_combined_1.to_csv('../combined_csvs/alvaro_combined.csv', index=False)

##### READING IN COMBINED CSVS

In [34]:
combined_df = pd.read_csv('../combined_csvs/combined_vg_sales_summary.csv')

In [35]:
# New dataframe containing data for preprocessing
df_new = combined_df[['Genre','Summary']].copy()
df_new

,Genre,Summary
0,Sports,Wii Sports was developed by Nintendo to show t...
1,Platform,"For more than 25 years, the Super Mario series..."
2,Racing,Mario Kart Wii includes 16 new courses and 16 ...
3,Shooter,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...
4,Sports,"Wii Sports Resort is an immersive, expansive a..."
...,...,...
18854,Simulation,
18855,Shooter,"Infiltrate terrorists' positions, acquire crit..."
18856,Fighting,
18857,Adventure,


In [36]:
df_new['Genre'].value_counts()

Action              2786
Sports              2394
Misc                2123
Adventure           1789
Role-Playing        1728
Shooter             1475
Racing              1359
Simulation          1065
Platform            1061
Fighting             897
Strategy             831
Puzzle               706
Action-Adventure     316
Music                131
Visual Novel         110
MMO                   43
Party                 36
Sandbox                4
Education              3
Board Game             2
Name: Genre, dtype: int64

In [37]:
# combine genres with low counts into misc
# combine repeating genres, Action and Action
df_new.replace({'Genre': 
             {'Board Game':'Misc', 
              'Party':'Misc',
              'MMO': 'Misc',
              'Sandbox':'Misc',
              'Education':'Misc'
#               'Music':'Misc'
#               'Adventure':'Action-Adventure', 
#               'Action':'Action-Adventure'
             }}, 
            inplace= True)
df_new['Genre'].value_counts()

Action              2786
Sports              2394
Misc                2211
Adventure           1789
Role-Playing        1728
Shooter             1475
Racing              1359
Simulation          1065
Platform            1061
Fighting             897
Strategy             831
Puzzle               706
Action-Adventure     316
Music                131
Visual Novel         110
Name: Genre, dtype: int64

In [38]:
# encode genres as a value in a new column
df_new['Genre_ID'] = df_new['Genre'].factorize()[0]
genre_df = df_new[['Genre', 'Genre_ID']].drop_duplicates()
# genre_df
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18859 entries, 0 to 18858
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Genre     18859 non-null  object
 1   Summary   18859 non-null  object
 2   Genre_ID  18859 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 442.1+ KB


In [39]:
# fig = plt.figure(figsize=(8,6))
# df_new.groupby('Genre').Summary.count().sort_values().plot.barh(
#     ylim=0, color='cornflowerblue', title= 'Genres for Each Summary\n')
# plt.xlabel('Summaries per Genre', fontsize = 10);

In [40]:
tfidf = TfidfVectorizer(sublinear_tf=True,
                        #ngram_range=(1, 4), 
                        stop_words='english')

# We transform each complaint into a vector
features = tfidf.fit_transform(df_new['Summary']).toarray()
labels = df_new['Genre_ID']


In [41]:
features.shape

(18859, 39990)

In [42]:
genre_to_id = dict(genre_df.values)
genre_to_id

{'Sports': 0,
 'Platform': 1,
 'Racing': 2,
 'Shooter': 3,
 'Role-Playing': 4,
 'Puzzle': 5,
 'Misc': 6,
 'Simulation': 7,
 'Action': 8,
 'Action-Adventure': 9,
 'Fighting': 10,
 'Strategy': 11,
 'Adventure': 12,
 'Music': 13,
 'Visual Novel': 14}

In [43]:
id_to_genre = dict(genre_df[['Genre_ID', 'Genre']].values)
id_to_genre

{0: 'Sports',
 1: 'Platform',
 2: 'Racing',
 3: 'Shooter',
 4: 'Role-Playing',
 5: 'Puzzle',
 6: 'Misc',
 7: 'Simulation',
 8: 'Action',
 9: 'Action-Adventure',
 10: 'Fighting',
 11: 'Strategy',
 12: 'Adventure',
 13: 'Music',
 14: 'Visual Novel'}

In [44]:
# extracting the most common single and double word-phrases (biigram) in each genre
N = 40
## wordcloud = []

for genre, genre_id in sorted(genre_to_id.items()):
    
    
    unigram = {}
##     bigram = {}
##     trigram = {}
    
    features_chi2 = chi2(features, labels == genre_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
##     data[genre] = b=unigrams
##     bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
##     data[genre] = bigrams
##     trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
##     data[genre] = trigrams
        
    print("\nGenre: %s" %(genre))
    print("  * Unigram keywords are: %s" %(', '.join(unigrams[-N:])))
##     print("  * Bigram keywords are: %s" %(', '.join(bigrams[-N:])))
##     print("  * Trigram keywords are: %s" %(', '.join(trigrams[-N:])))
##     wordcloud.append(data)



Genre: Action
  * Unigram keywords are: abilities, ghostbusters, voldemort, omnitrix, fu, stealth, kratos, film, incredibles, racing, assassins, comic, avengers, movie, platinumgames, thor, claire, redfield, films, evil, city, attacks, hulk, action, wolverine, marvel, dante, saints, resident, musou, assassin, transformers, bakugan, ninja, combat, spider, samurai, warriors, enemies, lego

Genre: Action-Adventure
  * Unigram keywords are: 3po, phasma, hux, dameron, kylo, poe, mistakenly, massachusetts, justcause, hopelessly, horseman, prematurely, blackgate, citizen, retold, laval, kenway, luthor, medici, flatbush, gulch, cuckoo, underprepared, darksiders, impa, triads, bilbo, doomstruction, doomstation, batman, spidey, zeroes, salem, bombur, thorin, ronan, hobbit, lego, superchargers, emmet

Genre: Adventure
  * Unigram keywords are: myst, seaworld, indiana, shamu, clannad, harry, evidence, witnesses, crime, sherlock, tantei, ncis, zelda, nancy, murder, d3, holmes, megamind, detective,

In [46]:
X = df_new['Summary'] # Collection of documents
y = df_new['Genre'] # Target or the labels we want to predict (i.e., the 15 different genres)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30,
                                                    random_state = 0)

In [50]:
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    #LinearSVC(),
    MultinomialNB(),
    #LogisticRegression(random_state=0),
]

# 5 Cross-validation
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

C:\Users\a_mcr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\a_mcr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\a_mcr\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 304, in fit
    accept_sparse="csc", dtype=DTYPE)
  File "C:\Users\a_mcr\anaconda3\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\a_mcr\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 72, in inner_f
    return f(**kwargs)
  File "C:\Users\a_mcr\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 802, in check_X_y
    estimator=estimator)
  File "C:\Users\a_mcr\anaconda3\lib\site-package

In [ ]:
X_train, X_test, y_train, y_test,indices_train,indices_test = train_test_split(features, 
                                                               labels, 
                                                               df2.index, test_size=0.30, 
                                                               random_state=1)
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)